## waste management

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# !pip install python-dotenv

In [3]:
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
# !pip install pandas

### Source Inventory Data: 
Contains information about stock levels at different source DCs, SKU batches, and their remaining shelf life.
### Destination Demand Data: 
Contains demand data for SKUs at different destination DCs along with per-unit prices.
### Transportation Cost Data: 
Includes per-unit transportation cost and transit time between source and destination DCs.

In [19]:
import pandas as pd

demand_file_path = ('/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/updated_destination_demand.csv') # insert the path of the csv file
demand_data = pd.read_csv(demand_file_path)

source_file_path = ('/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/updated_source_inventory.csv') # insert the path of the csv file
source_data = pd.read_csv(source_file_path)

transport_file_path = ('/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/transportation_costs.csv') # insert the path of the csv file
transport_data = pd.read_csv(transport_file_path)

In [20]:
#preview the csv file
demand_data.head()

,Destination_DC,SKU,Demand
0,DC_1,SKU_1,287
1,DC_1,SKU_2,316
2,DC_1,SKU_3,104
3,DC_1,SKU_4,66
4,DC_1,SKU_5,257


In [21]:
#preview the csv file
source_data.head()

,Source_DC,SKU,Batch_ID,AvailableStock,ShelfLifeDays,PerUnitPrice
0,DC_1,SKU_1,SKU_1_Batch_1,398,17,8.402283
1,DC_1,SKU_1,SKU_1_Batch_2,156,10,2.825307
2,DC_1,SKU_1,SKU_1_Batch_3,238,23,8.214560
3,DC_1,SKU_1,SKU_1_Batch_4,152,28,2.046000
4,DC_1,SKU_2,SKU_2_Batch_1,264,13,3.149458


In [22]:
#preview the csv file
transport_data.head()

,Source_DC,Destination_DC,SKU,PerUnitTransportationCost,TransitTimeDays
0,DC_1,DC_1,SKU_1,2.668404,1
1,DC_1,DC_1,SKU_2,3.206992,4
2,DC_1,DC_1,SKU_3,3.344718,5
3,DC_1,DC_1,SKU_4,3.020413,1
4,DC_1,DC_1,SKU_5,2.812345,9


In [ ]:
# !pip install langchain-community

  Using cached SQLAlchemy-2.0.38-cp310-cp310-macosx_10_9_x86_64.whl.metadata (9.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata (2.1 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
